<a href="https://colab.research.google.com/github/elisaboccini/AMDproject/blob/main/MBA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Market Basket Analysis**

Kaggle API

In [1]:
from google.colab import files
files.upload() #upload kaggle.json

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"elisaboccini","key":"3024605213579b61d22c8ab4775d25f4"}'}

In [4]:
#kaggle API 
!pip install -q kaggle
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
#!ls ~/.kaggle
!chmod 600 /root/.kaggle/kaggle.json
!kaggle datasets download -q ashirwadsangwan/imdb-dataset
!unzip imdb-dataset.zip

Archive:  imdb-dataset.zip
  inflating: name.basics.tsv.gz      
  inflating: name.basics.tsv/name.basics.tsv  
  inflating: title.akas.tsv.gz       
  inflating: title.akas.tsv/title.akas.tsv  
  inflating: title.basics.tsv.gz     
  inflating: title.basics.tsv/title.basics.tsv  
  inflating: title.principals.tsv.gz  
  inflating: title.principals.tsv/title.principals.tsv  
  inflating: title.ratings.tsv.gz    
  inflating: title.ratings.tsv/title.ratings.tsv  


Spark Session

In [5]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://www-us.apache.org/dist/spark/spark-3.1.1/spark-3.1.1-bin-hadoop2.7.tgz 
!tar xf spark-3.1.1-bin-hadoop2.7.tgz
!pip install -q findspark
!pip install pyspark

     |████████████████████████████████| 212.3MB 77kB/s 
     |████████████████████████████████| 204kB 48.3MB/s 
  Created wheel for pyspark: filename=pyspark-3.1.1-py2.py3-none-any.whl size=212767604 sha256=648d5489c59acc94d980072ac150b3165d24aafb38c8336bf1afa8252caefff8
  Stored in directory: /root/.cache/pip/wheels/0b/90/c0/01de724414ef122bd05f056541fb6a0ecf47c7ca655f8b3c0f
Successfully built pyspark


In [6]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.1.1-bin-hadoop2.7"

In [7]:
import findspark
findspark.init('spark-3.1.1-bin-hadoop2.7')

In [8]:
from pyspark import SparkConf, SparkContext
from pyspark.sql import SparkSession

In [9]:
spark = SparkSession.builder.enableHiveSupport().getOrCreate()

In [10]:
type(spark)

pyspark.sql.session.SparkSession

Dataset

In [11]:
#actors-movies junction
principals = spark.read.csv('title.principals.tsv.gz', sep="\t", header=True, inferSchema=True)

#for movies' titles
titlebasics = spark.read.csv('title.basics.tsv.gz', sep="\t", header=True, inferSchema=True)

#for actors' names
namesbasics = spark.read.csv('name.basics.tsv.gz', sep="\t", header=True, inferSchema=True)

In [12]:
principals.createOrReplaceTempView("principals")
titlebasics.createOrReplaceTempView("titlebasics")
namesbasics.createOrReplaceTempView("namesbasics")

In [13]:
# drop duplicates
titlebasics.dropDuplicates((['tconst'])).count() # 6326545
namesbasics.dropDuplicates((['nconst'])).count() #9711022

9711022

Exploratory Analysis

In [14]:
#in titlebasics not only movies are stored
spark.sql('''SELECT distinct (titleType)
              FROM titlebasics''').show()

+------------+
|   titleType|
+------------+
|    tvSeries|
|tvMiniSeries|
|     tvMovie|
|   tvEpisode|
|       movie|
|   tvSpecial|
|       video|
|   videoGame|
|     tvShort|
|       short|
+------------+



In [15]:
#Considering only actors (not crew) and only movies and shorts (not tvSeries, tvMiniSeries, tvEpisode, tvSpecial,video, videogames)
data = spark.sql('''SELECT tb.tconst, tb.primaryTitle, nb.primaryName, nb.nconst 
                          FROM titlebasics as tb LEFT JOIN principals as p ON tb.tconst = p.tconst INNER JOIN namesbasics as nb ON p.nconst = nb.nconst
                          WHERE ((p.category = 'actor' OR p.category = 'actress') AND (tb.titleType = 'movie' OR tb.titleType = 'tvMovie' OR tb.titleType = 'short' OR tb.titleType = 'tvShort')) ''')

data.createOrReplaceTempView('data')
data.show(5)
data.count() #3686467
# a more manageable dataset is obtained: the original principals table stores 14 mln records

+---------+-----------------+--------------+---------+
|   tconst|     primaryTitle|   primaryName|   nconst|
+---------+-----------------+--------------+---------+
|tt0083109| The Cabbage Soup|Louis de Funès|nm0000086|
|tt0062120|The Little Bather|Louis de Funès|nm0000086|
|tt0057422|    Squeak-squeak|Louis de Funès|nm0000086|
|tt0043691|     The Gamblers|Louis de Funès|nm0000086|
|tt0335439| Station mondaine|Louis de Funès|nm0000086|
+---------+-----------------+--------------+---------+
only showing top 5 rows



3686467

In [16]:
#Handling null values
spark.sql('''SELECT COUNT (tconst)
              FROM data
              WHERE (tconst = '/N') OR (primaryTitle = '/N') OR (primaryName = '/N') OR (nconst = '/N')''').show()

+-------------+
|count(tconst)|
+-------------+
|            0|
+-------------+



In [17]:
#Counting all the different movies
spark.sql('''select count(distinct tconst) 
            from data''').show() 
# 974512 

+----------------------+
|count(DISTINCT tconst)|
+----------------------+
|                974512|
+----------------------+



In [18]:
#Counting all the different primaryTitles

spark.sql('''select count(distinct primaryTitle) 
            from data''').show() 
#756111
#This means there are homonyms in movies and shorts and therefore the baskets can not be created 
#grouping by primaryTitle

+----------------------------+
|count(DISTINCT primaryTitle)|
+----------------------------+
|                      756111|
+----------------------------+



FP-Growth: pre-processing

In [19]:
#baskets creation
from pyspark.sql.functions import collect_set

moviesasbaskets = data.groupBy('tconst','primaryTitle').agg(collect_set('primaryName').alias('cast')) 
moviesasbaskets.createOrReplaceTempView('moviesasbaskets')

moviesasbaskets.show(n=5)
moviesasbaskets.count() 
#974512 
#A basket for each tconst as wanted

+---------+--------------------+--------------------+
|   tconst|        primaryTitle|                cast|
+---------+--------------------+--------------------+
|tt0000007|Corbett and Court...|[James J. Corbett...|
|tt0000378|          Les vagues|[Julie Souplet, G...|
|tt0000621|   That Fatal Sneeze|[Gertie Potter, T...|
|tt0000653|A Calamitous Elop...|[John R. Cumpson,...|
|tt0000841|       The Day After|[Marion Leonard, ...|
+---------+--------------------+--------------------+
only showing top 5 rows



974512

In [20]:
#Concatenating using hyphen
# in order to keep the info about the title and not only the id
from pyspark.sql import functions as f
moviesasbaskets = moviesasbaskets.withColumn('movie', f.concat(f.col('tconst'),f.lit('-'),f.col('primaryTitle')))

moviesasbaskets = moviesasbaskets.select('movie', 'cast')
moviesasbaskets.show(5)

+--------------------+--------------------+
|               movie|                cast|
+--------------------+--------------------+
|tt0000007-Corbett...|[James J. Corbett...|
|tt0000378-Les vagues|[Julie Souplet, G...|
|tt0000621-That Fa...|[Gertie Potter, T...|
|tt0000653-A Calam...|[John R. Cumpson,...|
|tt0000841-The Day...|[Marion Leonard, ...|
+--------------------+--------------------+
only showing top 5 rows



FP-Growth: Model implementation

In [21]:
# 0.0003 min Supp

#excluding episodes in tvSeries, it is reasonable to consider frequent an actor if it appears in  292 films 
#(distinct tconst = 974512)*0.0003

from pyspark.ml.fpm import FPGrowth
fpGrowth = FPGrowth(itemsCol="cast", minSupport=0.0003, minConfidence=0.6)
modelFP = fpGrowth.fit(moviesasbaskets) 

FP-Growth: Results

In [22]:
#Frequent itemsets
from pyspark.sql.functions import col, asc,desc
freqItemsets = modelFP.freqItemsets
freqItemsets.createOrReplaceTempView("freqItemsets")
freqItemsets.orderBy(col('freq').desc()).show()

+--------------------+----+
|               items|freq|
+--------------------+----+
|         [Mel Blanc]|1030|
|  [Matsunosuke Onoe]| 932|
|      [Brahmanandam]| 797|
|       [Adoor Bhasi]| 585|
|      [Eddie Garcia]| 510|
|       [Jack Mercer]| 453|
|        [Prem Nazir]| 436|
|   [Jack Richardson]| 434|
|         [Lee Moran]| 429|
|      [Sung-il Shin]| 412|
|       [Eddie Lyons]| 406|
|      [Paquito Diaz]| 393|
|      [Oliver Hardy]| 392|
|  [Masayoshi Nogami]| 390|
|[J. Warren Kerrigan]| 383|
|         [Mammootty]| 381|
|    [Aachi Manorama]| 356|
|           [Bahadur]| 348|
|          [Mohanlal]| 346|
|[Gilbert M. 'Bron...| 342|
+--------------------+----+
only showing top 20 rows



In [23]:
#Association rules
associationRules = modelFP.associationRules
associationRules.createOrReplaceTempView("associationRules")
associationRules.show(n=5) 

+-------------+-------------+------------------+------------------+--------------------+
|   antecedent|   consequent|        confidence|              lift|             support|
+-------------+-------------+------------------+------------------+--------------------+
|[Eddie Lyons]|  [Lee Moran]|0.7832512315270936|1779.2254641909815|3.263171720820267E-4|
|  [Lee Moran]|[Eddie Lyons]|0.7412587412587412|1779.2254641909813|3.263171720820267E-4|
+-------------+-------------+------------------+------------------+--------------------+



In [25]:
# transform examines the input items against all the association rules and summarize
# the consequents as prediction
association = modelFP.transform(moviesasbaskets)
association.createOrReplaceTempView('association')
association.show()

+--------------------+--------------------+----------+
|               movie|                cast|prediction|
+--------------------+--------------------+----------+
|tt0000007-Corbett...|[James J. Corbett...|        []|
|tt0000378-Les vagues|[Julie Souplet, G...|        []|
|tt0000621-That Fa...|[Gertie Potter, T...|        []|
|tt0000653-A Calam...|[John R. Cumpson,...|        []|
|tt0000841-The Day...|[Marion Leonard, ...|        []|
|tt0001628-The Fou...|    [Norma Talmadge]|        []|
|tt0001693-The Imm...|[Francis Ford, Wi...|        []|
|tt0001820-The Out...|[Gilbert M. 'Bron...|        []|
|tt0002453-The Rec...|[Harold Lockwood,...|        []|
|tt0002870-Flood Tide|[Marie Eline, Lil...|        []|
|tt0003172-The Mou...|[Gladden James, E...|        []|
|tt0003404-The Sta...|[Theodor Burghard...|        []|
|tt0003601-The Adv...|[Joe Evans, Fred ...|        []|
|tt0003649-Back to...|[Oliver Hardy, Ma...|        []|
|tt0003687-Bombs a...|[Rube Miller, Ali...|        []|
|tt0003744

In [26]:
spark.sql("""select *
             from association
             where size(prediction) >= 1 """).show()

+--------------------+--------------------+-------------+
|               movie|                cast|   prediction|
+--------------------+--------------------+-------------+
|tt0256990-The Par...|[William Bertram,...|  [Lee Moran]|
|tt0339764-The Tan...|[Harry A. Pollard...|  [Lee Moran]|
|tt8249264-Just in...|[Beth Darlington,...|  [Lee Moran]|
|tt0329718-Upper a...|[Alberta Vaughn, ...|[Eddie Lyons]|
|tt0349292-Cupid's...|[Louise Glaum, Ed...|  [Lee Moran]|
|tt0332627-Game Hu...|[Queenie the Hors...|[Eddie Lyons]|
|tt1318984-Three o...|[Dick Dewell, Jac...|[Eddie Lyons]|
|tt0256646-Better ...|[Margarita Fische...|  [Lee Moran]|
|tt0249766-A Lucky...|       [Eddie Lyons]|  [Lee Moran]|
|tt0334494-Their T...|[Page Peters, Art...|[Eddie Lyons]|
|tt0332698-A Man o...|[Stella Adams, Do...|[Eddie Lyons]|
|tt1542726-Pickin'...|[Doris McMahon, B...|[Eddie Lyons]|
|tt0334724-Women F...|[Alberta Vaughn, ...|[Eddie Lyons]|
|tt0349256-The Cou...|[Louise Glaum, Ed...|  [Lee Moran]|
|tt0860492-Onc

In [27]:
#Most frequent actors
mostfrequentactor= spark.sql("""select items, freq
                                    from freqItemsets
                                    where size(items) = 1
                                    order by freq desc""")
mostfrequentactor.show(n=5)
mostfrequentactor.count()

+------------------+----+
|             items|freq|
+------------------+----+
|       [Mel Blanc]|1030|
|[Matsunosuke Onoe]| 932|
|    [Brahmanandam]| 797|
|     [Adoor Bhasi]| 585|
|    [Eddie Garcia]| 510|
+------------------+----+
only showing top 5 rows



41

In [28]:
#Itemsets having size = 2
mostfrequent2actors= spark.sql("""select items, freq
                                      from freqItemsets
                                      where size(items) = 2
                                      order by freq desc""")
mostfrequent2actors.show(n=5)
mostfrequent2actors.count()

+--------------------+----+
|               items|freq|
+--------------------+----+
|[Eddie Lyons, Lee...| 318|
+--------------------+----+



1

In [30]:
#Lowering the threshold

# 0.0001 min Supp
fpGrowth1 = FPGrowth(itemsCol="cast", minSupport=0.0001, minConfidence=0.6)
modelFP1 = fpGrowth1.fit(moviesasbaskets) 

In [31]:
#Frequent itemsets
freqItemsets1 = modelFP1.freqItemsets
freqItemsets1.createOrReplaceTempView("freqItemsets1")
freqItemsets1.orderBy(col('freq').desc()).show()

+--------------------+----+
|               items|freq|
+--------------------+----+
|         [Mel Blanc]|1030|
|  [Matsunosuke Onoe]| 932|
|      [Brahmanandam]| 797|
|       [Adoor Bhasi]| 585|
|      [Eddie Garcia]| 510|
|       [Jack Mercer]| 453|
|        [Prem Nazir]| 436|
|   [Jack Richardson]| 434|
|         [Lee Moran]| 429|
|      [Sung-il Shin]| 412|
|       [Eddie Lyons]| 406|
|      [Paquito Diaz]| 393|
|      [Oliver Hardy]| 392|
|  [Masayoshi Nogami]| 390|
|[J. Warren Kerrigan]| 383|
|         [Mammootty]| 381|
|    [Aachi Manorama]| 356|
|           [Bahadur]| 348|
|          [Mohanlal]| 346|
|[Gilbert M. 'Bron...| 342|
+--------------------+----+
only showing top 20 rows



In [32]:
#Association rules
associationRules1 = modelFP1.associationRules
associationRules1.createOrReplaceTempView("associationRules1")
associationRules1.show(n=5) 

+--------------------+-------------------+------------------+------------------+--------------------+
|          antecedent|         consequent|        confidence|              lift|             support|
+--------------------+-------------------+------------------+------------------+--------------------+
|[Curly Howard, La...|       [Moe Howard]|               1.0|4490.8387096774195|1.067200814356313...|
|      [Larry Storch]|  [Norma MacMillan]|           0.78125|        6797.65625|1.026154629188763...|
|[Kijaku Ôtani, Ma...| [Kitsuraku Arashi]| 0.743421052631579| 5406.512961508249|1.159554730983302...|
|[Kijaku Ôtani, Ma...|[Suminojo Ichikawa]|0.6776315789473685|  5368.78134360291|1.056939268064426E-4|
|        [Larry Fine]|       [Moe Howard]|0.9906103286384976| 4448.671210056035|2.165186267588290...|
+--------------------+-------------------+------------------+------------------+--------------------+
only showing top 5 rows



In [34]:
#filtering association rules having confidence equal to 1
spark.sql("""select (*)
           from associationRules1
           where confidence = '1.0' """).show()

+--------------------+------------+----------+------------------+--------------------+
|          antecedent|  consequent|confidence|              lift|             support|
+--------------------+------------+----------+------------------+--------------------+
|[Curly Howard, La...|[Moe Howard]|       1.0|4490.8387096774195|1.067200814356313...|
|[Curly Howard, Mo...|[Larry Fine]|       1.0| 4575.173708920188|1.067200814356313...|
+--------------------+------------+----------+------------------+--------------------+



In [36]:
spark.sql("""select antecedent as `IF`, consequent as `THEN`, confidence, lift
              from associationRules1
              where lift > 0
              order by confidence desc
              limit 20""").show()

+--------------------+--------------------+------------------+------------------+
|                  IF|                THEN|        confidence|              lift|
+--------------------+--------------------+------------------+------------------+
|[Curly Howard, La...|        [Moe Howard]|               1.0|4490.8387096774195|
|[Curly Howard, Mo...|        [Larry Fine]|               1.0| 4575.173708920188|
|[Pauline Bush, Ja...|[J. Warren Kerrigan]|0.9919354838709677| 2523.898256548471|
|        [Larry Fine]|        [Moe Howard]|0.9906103286384976| 4448.671210056035|
|      [Curly Howard]|        [Larry Fine]|0.9904761904761905| 4531.600625978092|
|      [Curly Howard]|        [Moe Howard]|0.9904761904761905| 4448.068817204301|
|[Bebe Daniels, 'S...|      [Harold Lloyd]|0.9863945578231292| 4666.278317152103|
|[Harold Lloyd, 'S...|      [Bebe Daniels]|0.9863945578231292| 4216.023391812865|
|[Kitsuraku Arashi...|  [Matsunosuke Onoe]|0.9826086956521739|1027.4291472289606|
|[Suminojo Ichik

In [37]:
#Most frequent actors
mostfrequentactor1= spark.sql("""select items, freq
                                    from freqItemsets1
                                    where size(items) = 1
                                    order by freq desc""")
mostfrequentactor1.show(n=5)
mostfrequentactor1.count()

+------------------+----+
|             items|freq|
+------------------+----+
|       [Mel Blanc]|1030|
|[Matsunosuke Onoe]| 932|
|    [Brahmanandam]| 797|
|     [Adoor Bhasi]| 585|
|    [Eddie Garcia]| 510|
+------------------+----+
only showing top 5 rows



1342

In [38]:
#Itemsets having size = 2
mostfrequent2actors1= spark.sql("""select items, freq
                                      from freqItemsets1
                                      where size(items) = 2
                                      order by freq desc""")
mostfrequent2actors1.show(n=5)
mostfrequent2actors1.count()

+--------------------+----+
|               items|freq|
+--------------------+----+
|[Eddie Lyons, Lee...| 318|
|[Prem Nazir, Adoo...| 236|
|[Mae Questel, Jac...| 216|
|[Larry Fine, Moe ...| 211|
|[J. Warren Kerrig...| 186|
+--------------------+----+
only showing top 5 rows



56

In [39]:
#Itemsets of size = 3
mostfrequent3actors1= spark.sql("""select items, freq
                                      from freqItemsets1
                                      where size(items) = 3
                                      order by freq desc""")
mostfrequent3actors1.show()
mostfrequent3actors1.count() 

+--------------------+----+
|               items|freq|
+--------------------+----+
|[Harold Lloyd, Be...| 145|
|[Pauline Bush, J....| 123|
|[Milburn Morante,...| 119|
|[Kitsuraku Arashi...| 113|
|[Winifred Greenwo...| 112|
|[Curly Howard, La...| 104|
|[Vivian Rich, Lou...| 104|
|[Suminojo Ichikaw...| 103|
|[Lillian Peacock,...| 101|
+--------------------+----+



9

In [41]:
spark.sql("""select items
              from freqItemsets1
              where size(items) = 3""").toPandas()

,items
0,"[Kitsuraku Arashi, Kijaku Ôtani, Matsunosuke O..."
1,"[Lillian Peacock, Gale Henry, Billy Franey]"
2,"[Pauline Bush, J. Warren Kerrigan, Jack Richar..."
3,"[Curly Howard, Larry Fine, Moe Howard]"
4,"[Vivian Rich, Louise Lester, Jack Richardson]"
5,"[Milburn Morante, Gale Henry, Billy Franey]"
6,"[Suminojo Ichikawa, Kijaku Ôtani, Matsunosuke ..."
7,"[Winifred Greenwood, George Field, Edward Coxen]"
8,"[Harold Lloyd, Bebe Daniels, 'Snub' Pollard]"


In [ ]:
#spark.stop()